In [58]:
%reload_ext autoreload
%autoreload 2

import os
os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import math
import numpy as np
import pandas as pd
import datetime
import pytz
import seaborn as sns
from datetime import datetime, timedelta
import pickle

from watttime import WattTimeForecast, WattTimeHistorical

import optimizer.s3 as s3u
import evaluation.eval_framework as efu

import watttime.shared_anniez.alg.optCharger as optC
import watttime.shared_anniez.alg.moer as Moer


username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

actual_data = WattTimeHistorical(username, password)
hist_data = WattTimeForecast(username, password)

s3 = s3u.s3_utils()
key = '20240726_1k_synth_users_163_days.csv'
generated_data = s3.load_csvdataframe(file=key)

In [61]:
# Synthetic user data

region = "CAISO_NORTH"

synth_data = generated_data.copy(deep=True)
synth_data["plug_in_time"] = pd.to_datetime(synth_data["plug_in_time"])
synth_data["unplug_time"] = pd.to_datetime(synth_data["unplug_time"])

# Cached version of the get_*_data functions

In [60]:

HISTORICAL_ACTUAL_CACHE = pickle.loads(s3u.s3_utils().load_file(f"{region}_actual.pkl"))
HISTORICAL_FORECAST_CACHE = pickle.loads(s3u.s3_utils().load_file(f"{region}_fore.pkl"))

---

In [62]:
def get_historical_fcst_data_cached(plug_in_time, horizon, region):
    time_zone = efu.get_timezone_from_dict(region)
    plug_in_time_utc = pd.Timestamp(efu.convert_to_utc(plug_in_time, time_zone))
    date = plug_in_time.date()
    if (region, date) not in HISTORICAL_FORECAST_CACHE.keys():
        print(type(date), date)
        start = pd.to_datetime(date)
        end = (pd.to_datetime(date) + pd.Timedelta("1d"))      
        HISTORICAL_FORECAST_CACHE[(region, date)] = hist_data.get_historical_forecast_pandas(
            start - pd.Timedelta("9h"),
            end + pd.Timedelta("9h"),
            region,
        )
    cache = HISTORICAL_FORECAST_CACHE[(region, date)]
        
    # make this match efu.get_historical_fsct_data
    generated_at_times = cache["generated_at"].unique()
    generated_at = max([t for t in generated_at_times if t < plug_in_time_utc])
    df = cache[cache["generated_at"] == generated_at].copy()
    return df.iloc[:math.ceil(horizon / 12) * 12]

def get_historical_actual_data_cached(plug_in_time, horizon, region):
    time_zone = efu.get_timezone_from_dict(region)
    plug_in_time_utc = pd.Timestamp(efu.convert_to_utc(plug_in_time, time_zone))
    date = plug_in_time.date()
    
    if (region, date) not in HISTORICAL_ACTUAL_CACHE.keys():
        start = pd.to_datetime(date)
        end = (pd.to_datetime(date) + pd.Timedelta("2d"))   
        HISTORICAL_ACTUAL_CACHE[(region, date)] = actual_data.get_historical_pandas(
            start - pd.Timedelta("9h"),
            end + pd.Timedelta("9h"),
            region,
        )
    cache = HISTORICAL_ACTUAL_CACHE[(region, date)]

    t_start = max([t for t in cache["point_time"].unique() if t < plug_in_time_utc])
    df =  cache[cache["point_time"] >= t_start].copy()
    return df.iloc[:math.ceil(horizon / 12) * 12 + 1].reset_index(drop=True)
    

## Generate results

In [63]:
def get_total_emission(moer, schedule): 
    x = np.array(schedule).flatten()
    return np.dot(moer[:x.shape[0]], x)

def generate_results(synth_data):
    print("Get the data: forecast")
    synth_data['moer_data'] = synth_data.apply(
        lambda x: get_historical_fcst_data_cached(
        x.plug_in_time,
        math.ceil(x.total_intervals_plugged_in),
        region = region
        ),
        axis = 1
    )

    print("Get the data: actual")
    synth_data['moer_data_actual'] = synth_data.apply(
        lambda x: get_historical_actual_data_cached(
            x.plug_in_time,
            math.ceil(x.total_intervals_plugged_in),
            region = region
        ),
        axis = 1
    )

    print("Generate the baeline schedule training on the actual")
    synth_data['charger_baseline_actual'] = synth_data.apply(
        lambda x: efu.get_schedule_and_cost(
            x.MWh_fraction,
            x.charged_kWh_actual / 1000,
            math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
            x.moer_data_actual,
            asap = True
        ),
        axis = 1
    )
    print("Get the baseline schedule")
    synth_data['baseline_charging_schedule'] = synth_data['charger_baseline_actual'].apply(
        lambda  x: x.get_schedule()
    )

    print("Get the baseline actual emissions")
    synth_data['baseline_actual_emissions'] = synth_data['charger_baseline_actual'].apply(
        lambda  x: x.get_total_emission()
    )
        
    print("Generate the simple schedule, training on the forecast")
    synth_data['charger_simple_forecast']= synth_data.apply(
        lambda x: efu.get_schedule_and_cost(
            x.MWh_fraction,
            x.charged_kWh_actual / 1000,
            math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
            x.moer_data,
            asap = False
        ),
        axis = 1
    )
    synth_data['simple_charging_schedule'] = synth_data['charger_simple_forecast'].apply(
        lambda  x: x.get_schedule()
    )

    print("Get the estimated emissions, e.g. evaluating on the forecast")
    synth_data['simple_estimated_emissions'] = synth_data['charger_simple_forecast'].apply(
        lambda  x: x.get_total_emission()
    )

    print("Generate the simple schedule training on the actual: this is the ideal schedule")
    synth_data['charger_simple_actual']= synth_data.apply(
        lambda x: efu.get_schedule_and_cost(
            x.MWh_fraction,
            x.charged_kWh_actual / 1000,
            math.ceil(x.total_intervals_plugged_in), # will throw an error if the plug in time is too shart to reach full charge, should soften to a warning
            x.moer_data_actual,
            asap = False
        ), 
        axis = 1
    )

    synth_data['simple_actual_charging_schedule'] = synth_data['charger_simple_actual'].apply(
        lambda  x: x.get_schedule()
        )

    synth_data['simple_ideal_emissions'] = synth_data['charger_simple_actual'].apply(
        lambda  x: x.get_total_emission()
        )

    print("MOER - No Optimization - Actual Emissions")
    synth_data["simple_actual_emissions"] = synth_data.apply(
        lambda x: get_total_emission(
            x.moer_data_actual['value'],
            x.simple_charging_schedule,
        ),
        axis = 1
    )

    cols = [
        "user_type",
        "power_output_rate",
        "distinct_dates",
        "plug_in_time",
        "total_intervals_plugged_in",
        "charged_kWh_actual",
        "MWh_fraction",
        "simple_actual_emissions",
        "baseline_actual_emissions",
        "simple_estimated_emissions",
        "simple_ideal_emissions"
    ]

    return synth_data[cols]

# Generate results

In [65]:

%%capture 
# the %%capture cellmagic stops all the print messages being shown
results_data = generate_results(synth_data)

In [66]:
%%capture

synth_data_daytime = synth_data.copy()
synth_data_daytime["plug_in_time"] = synth_data_daytime["plug_in_time"] - pd.Timedelta(hours=12)

results_data_daytime =  generate_results(synth_data_daytime)

# Save results

In [67]:
s3.store_csvdataframe(
    results_data, f"results/20240818_1k_synth_users_163_days_{region}.csv"
)
s3.store_csvdataframe(ß
    results_data_daytime, f"results/20240818_1k_synth_users_163_days_{region}_daytime.csv"
)

Successful S3 put_object response. Status - 200
Successful S3 put_object response. Status - 200


# Analyze results

In [68]:
results_data["largest_possible_difference"] = results_data["simple_ideal_emissions"] - results_data["baseline_actual_emissions"]
print("Standard synthetic user data. Total possible emission savings:")
print(results_data["largest_possible_difference"].sum())

results_data_daytime["largest_possible_difference"] = results_data_daytime["simple_ideal_emissions"] - results_data_daytime["baseline_actual_emissions"]
print("Daytime version of synthetic user data. Total possible emission savings:")
print(results_data_daytime["largest_possible_difference"].sum())

Standard synthetic user data. Total possible emission savings:
-85483.50453249615
Daytime version of synthetic user data. Total possible emission savings:
-614162.1579068066
